In [1]:
# additional function
from skimage.feature import local_binary_pattern
from skimage.feature import hog
from skimage.io import imread
from sklearn.externals import joblib
# To read file names
import argparse as ap
import glob
import os
import cv2

#Normalize
#L1
#L1-sqrt
#L2
#L2-Hys

min_wdw_sz =[64, 128]
step_size = [8, 8]
orientations = 9
pixels_per_cell = (8, 8)
cells_per_block = (4, 4)
visualize = False
normalize = True


from sklearn import preprocessing
import numpy as np

def hogMy(im, orientations, pixels_per_cell, cells_per_block, visualize, normalize): 
    fd = hog(im, orientations=orientations, pixels_per_cell=pixels_per_cell, cells_per_block=cells_per_block, visualise=visualize,
            block_norm= "L1", transform_sqrt=False, feature_vector=True)
    hog_features = np.array(fd, 'float64')
    
    # Normalize the features
    pp = preprocessing.StandardScaler().fit(hog_features)
    hog_features = pp.transform(hog_features)
    
    return hog_features



In [1]:
import utilsDetect#; reload(utilsDetect)
from utilsDetect import *

In [3]:
sourceFile = "TA/shop1.jpg"

pos_source = "train/positive"
neg_source = "train/negative"

# 1. DataSet

In [9]:
from skimage.feature import local_binary_pattern
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.externals import joblib
import argparse as ap
import glob
import os


full_data=[]
full_label=[]

# Load the positive features
for feat_path in glob.glob(os.path.join(pos_source,"*")):
    im = imread(feat_path, as_grey=True)
    #im=resizeImage(im)
    fd = hogArray(im, orientations, pixels_per_cell, cells_per_block, visualize, normalize)
    full_data.append(fd)
    full_label.append(1)

# Load the negative features
for feat_path in glob.glob(os.path.join(neg_source,"*")):
    im = imread(feat_path, as_grey=True)
    im=resizeImage(im)
    fd = hogArray(im, orientations, pixels_per_cell, cells_per_block, visualize, normalize)
    full_data.append(fd)
    full_label.append(0) 

In [10]:
from matplotlib.colors import ListedColormap
from sklearn import datasets, linear_model, metrics, cross_validation 


train_data = []
train_label = []
test_data = []
test_label = []

#split data
train_data, test_data, train_label, test_label = cross_validation.train_test_split(
    full_data, full_label, test_size=0.3, random_state=1)

# Linear Classifier

In [25]:
model_path = "train/svmLinearClassifier.model"

#создаем классификатора
ridge_classifier = linear_model.RidgeClassifier(random_state=1)
# обучим модель для этого передадим обучающую выборку и метки классов
ridge_classifier.fit(train_data, train_label)
# теперь можем строить наши предсказания:
ridge_prediction=ridge_classifier.predict(test_data)

# формально оценим качество модели при помощи accuracy:
print('accuracy: {}'.format(metrics.accuracy_score(test_label, ridge_prediction)))
print('f1: {}'.format(metrics.f1_score(test_label, ridge_prediction)))
# кроме качества обучения  мы можем смотреть на веса признаков 
print (ridge_classifier.coef_)
# свободный коеффициент. Коэффициент перед свободным членом
print (ridge_classifier.intercept_)


#save model
joblib.dump(ridge_classifier, model_path)

accuracy: 0.8626506024096385
f1: 0.5488126649076517
[[ 0.01342673 -0.03837251 -0.0002492  ..., -0.01950641  0.03474442
   0.01409306]]
[-0.96757419]


['train/svmLinearClassifier.model']

# Liner Model Default

this model have got in example 

In [26]:
clf = LinearSVC()
print ("Training a Linear SVM Classifier")
clf.fit(train_data, train_label)
# теперь можем строить наши предсказания:
clf_prediction=clf.predict(test_data)
# формально оценим качество модели при помощи accuracy:
print('accuracy: {}'.format(metrics.accuracy_score(test_label, clf_prediction)))
print('f1: {}'.format(metrics.f1_score(test_label, clf_prediction)))

#save model
joblib.dump(clf, "train/svm.model")

Training a Linear SVM Classifier
accuracy: 0.9060240963855422
f1: 0.5682656826568266


['train/svm.model']

# Logistic regression

In [14]:
model_path = "train/svmLinearRegression.model"

log_regressor=linear_model.LogisticRegression(random_state=1)
#обучаем модель
log_regressor.fit(train_data, train_label)
#прогнозируем
log_predict=log_regressor.predict(test_data)
# regression вероятностный метод поэтому каждый ответ имеет некую вероятностную составляющую
log_predict_probability = log_regressor.predict_proba(test_data)
#оценим
print ('accuracy: {}'.format(metrics.accuracy_score(log_predict, test_label)))
print ('f1: {}'.format(metrics.f1_score(log_predict, test_label)))


#save model
joblib.dump(log_regressor, model_path)
print ("Regression saved to {}".format(model_path))

0.845896147404
0.406451612903
Regression saved to train/svmLinearRegression.model


# Random Forest

In [4]:
model_path = "train/svmRandomF.model"
#модель случайный лес
from sklearn.ensemble import RandomForestClassifier
from sklearn import pipeline, preprocessing
from sklearn import metrics 

#classifier = RandomForestClassifier(random_state=0, 
                               # max_depth=2, 
                               #   n_estimators=50)
classifier = RandomForestClassifier(random_state=0, 
                                    min_samples_split=5, 
                                    n_estimators=50)


#обучим
classifier.fit(train_data, train_label)

clf_prediction=classifier.predict(test_data)
# формально оценим качество модели при помощи accuracy:
print('accuracy: {}'.format(metrics.accuracy_score(test_label, clf_prediction)))
print('f1: {}'.format(metrics.f1_score(test_label, clf_prediction)))

#save model
joblib.dump(classifier, model_path)

accuracy: 0.9307253463732681
f1: 0.5502645502645502


['train/svmRandomF.model']

# Parametrize a model

In [19]:
# модель с параметрами по умолчанию
ClassifierDefault = linear_model.SGDClassifier(random_state=0)
ClassifierDefault.fit(train_data, train_label)
#прогнозируем
cd_predict=ClassifierDefault.predict(test_data)
#оценим
print ('accuracy: {}'.format(metrics.accuracy_score(cd_predict, test_label)))
print ('f1: {}'.format(metrics.f1_score(cd_predict, test_label)))

accuracy: 0.8850855745721271
f1: 0.5791044776119403


In [20]:
from sklearn import cross_validation, grid_search

#эту же модель настроим через сетку
ClassifierCustom = linear_model.SGDClassifier(random_state=0)
ClassifierCustom.get_params().keys()

dict_keys(['verbose', 'warm_start', 'l1_ratio', 'shuffle', 'eta0', 'n_jobs', 'random_state', 'fit_intercept', 'average', 'power_t', 'alpha', 'epsilon', 'penalty', 'class_weight', 'learning_rate', 'loss', 'n_iter'])

In [21]:
#create parametric matrix 
# по имеющимся в модели параметрам мы записываем возможные значения перебирая которые 
# определим лучшую модель методом перебора
parameters_grid ={
    'loss':['hinge', 'log', 'squared_hinge', 'squared_loss'],
    'penalty':['l1', 'l2'],
    'n_iter': [5, 6, 7, 8, 9, 10],
    'alpha': np.linspace(0.0001, 0.001, num=5),
}

#создаем стратегию cross-validation
cv=cross_validation.StratifiedShuffleSplit(train_label, test_size=0.2, n_iter=10, random_state=0)

#создаем объект gridsearch, который будет искать по метрике "ACCURACY" лучшие параметры
grid_cv=grid_search.GridSearchCV(ClassifierCustom, parameters_grid, scoring='accuracy', cv=cv)


In [22]:
%%time
grid_cv.fit(train_data, train_label)
# далее обучаем сетку

Wall time: 53min


GridSearchCV(cv=StratifiedShuffleSplit(labels=[0 0 ..., 0 0], n_iter=10, test_size=0.2, random_state=0),
       error_score='raise',
       estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=0, shuffle=True, verbose=0,
       warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'alpha': array([ 0.0001 ,  0.00032,  0.00055,  0.00078,  0.001  ]), 'n_iter': [5, 6, 7, 8, 9, 10], 'loss': ['hinge', 'log', 'squared_hinge', 'squared_loss'], 'penalty': ['l1', 'l2']},
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

In [28]:
# получить лучший классификатор:
grid_cv.best_estimator_

SGDClassifier(alpha=0.001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=10, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=0, shuffle=True, verbose=0,
       warm_start=False)

In [29]:
model_path="train/svmBestSGD.model"
#построим модель при этом возьмем те параметры которые
# мы уже подобрали в прошлую попытку как оптимальные при наших данных
# {'regression__n_iter': 3, 'regression__loss': 'squared_loss', 
# 'scaling__with_mean': 0, 'regression__alpha': 0.01, 
# 'regression__penalty': 'l2'}

bestSGD = linear_model.SGDClassifier(alpha=0.001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=10, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=0, shuffle=True, verbose=0,
       warm_start=False)
#обучим
bestSGD.fit(train_data, train_abel)
#прогнозируем
cd_predict=bestSGD.predict(test_data)
#оценим
print ('accuracy: {}'.format(metrics.accuracy_score(cd_predict, test_label)))
print ('f1: {}'.format(metrics.f1_score(cd_predict, test_label)))

#save model
joblib.dump(bestSGD, model_path)

accuracy: 0.9413654618473896
f1: 0.7474048442906573


['train/svmBestSGD.model']